In [2]:
import pandas as pd

In [133]:
# dataset
dataset = pd.read_csv('airflow/data/Dataset/data.csv', parse_dates=['date'])
dataset['period']=dataset['date'].dt.to_period('M')
dataset.sort_values(by="date" ,inplace=True)
one_month = list(set(dataset["period"]))[1]
dataset = dataset[dataset["period"]== one_month].transpose()
dataset.columns=[ i for i in range(1,len(dataset.columns)+1)]
df = dataset.transpose()

# topics indexing
topics = pd.read_csv('airflow/data/Outputs/topics.csv', index_col='doc_num')
topics_columns = []
i=0
for topic in topics.columns:
    topics_columns.append(['topic_' + str(i) , topic])
    i=i+1
topics_columns=pd.DataFrame(topics_columns,columns=['index','topic'])
topics.columns = topics_columns['index']

# appriori indexing
appriori = pd.read_csv('airflow/data/Outputs/appriori.csv')
appriori_columns = []
i=0
for topic in appriori.columns:
    appriori_columns.append(['bigramm_' + str(i) , topic])
    i = i + 1
appriori_columns = pd.DataFrame(appriori_columns,columns=['index','topic'])
appriori.columns = appriori_columns['index']

# indexing trends
trends = pd.read_csv('airflow/data/Outputs/trends.csv')
trends.fillna(0, inplace=True)
trends_columns = []
i=0
for topic in trends.columns:
    trends_columns.append(['trend_' + str(i) , topic])
    i = i + 1
trends_columns = pd.DataFrame(trends_columns,columns=['index','topic'])
trends.columns = trends_columns['index']

In [134]:
trends.head(2)

index,trend_0,trend_1,trend_2,trend_3,trend_4
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0


In [138]:
(topics.iloc[3])

index
topic_0    0.026187
topic_1    0.000000
topic_2    0.258143
topic_3    0.035583
topic_4    0.010990
topic_5    0.000000
topic_6    0.004084
topic_7    0.002550
topic_8    0.000000
topic_9    0.000000
Name: 3, dtype: float64

In [136]:
appriori.head(2)

index,bigramm_0,bigramm_1,bigramm_2,bigramm_3,bigramm_4,bigramm_5,bigramm_6,bigramm_7,bigramm_8,bigramm_9,...,bigramm_40894,bigramm_40895,bigramm_40896,bigramm_40897,bigramm_40898,bigramm_40899,bigramm_40900,bigramm_40901,bigramm_40902,bigramm_40903
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
import psycopg2 as pg2

# Connect to database
conn = pg2.connect(host='localhost', database='optionsdata', user='openoptions', password='openoptions', port='5432')

In [153]:
conn.status

1

In [154]:
s = ""
s += "SELECT"
s += " table_schema"
s += ", table_name"
s += " FROM information_schema.tables"
s += " WHERE"
s += " ("
s += " table_schema = 'public'"
s += " )"
s += " ORDER BY table_schema, table_name;"


In [155]:
db_cursor = conn.cursor()
db_cursor.execute(s)
list_tables = db_cursor.fetchall()

for t_name_table in list_tables:
    print(t_name_table)

('public', 'articles')
('public', 'topics')
('public', 'topics_columns')
('public', 'trends')
('public', 'trends_columns')


In [156]:
# query database
query = 'SELECT * FROM articles;'

In [157]:
cur = conn.cursor()
cur.execute(query)

ha = cur.fetchall()
print(type(ha))

<class 'list'>


In [21]:
import pandas as pd
df = pd.DataFrame(ha)

In [158]:
df.head()

,titre,text,date,link,img,period
1,Financement des entreprises : La banque allema...,"La banque allemande DEG, filiale du groupe KFW...",2022-05-01,https://lematin.ma/express/2022/financement-en...,https://static.lematin.ma/files/lematin/images...,2022-05
2,Le crédit bancaire termine le premier trimestr...,Le trend haussier du crédit bancaire s’est pou...,2022-05-01,https://lematin.ma/express/2022/credit-bancair...,https://static.lematin.ma/files/lematin/images...,2022-05
3,Financement de l’entrepreneuriat : les solutio...,"Que ce soit auprès des banques, via des progra...",2022-05-01,https://lematin.ma/express/2022/financement-le...,https://static.lematin.ma/files/lematin/images...,2022-05
4,Vers une chaîne d’approvisionnement agroalimen...,Les économies mondiales ont connu une déstabil...,2022-05-01,https://www.lavieeco.com/economie/vers-une-cha...,https://www.lavieeco.com/wp-content/uploads/20...,2022-05
5,La facture dématérialisée,La Loi de finances 2022 devrait donner un coup...,2022-05-01,https://www.lavieeco.com/economie/la-facture-d...,https://www.lavieeco.com/wp-content/uploads/20...,2022-05
